In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix, Rgba
import time
import pydrake.multibody.rational as rational_forward_kinematics
from pydrake.all import RationalForwardKinematics
from pydrake.all import GeometrySet, CollisionFilterDeclaration, CollisionFilterManager
from pydrake.geometry.optimization import IrisOptions, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions

In [4]:
from pydrake.geometry.optimization_dev import (CspaceFreePolytope, 
                                               SeparatingPlaneOrder)

# Build and set up the visualization the plant and the visualization of the C-space obstacle

We first set up a simple 2-DOF system and visualize both the plant and the configuration constraint.Click on the two links at the bottom to view the plant and the configuration space.

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [5]:
#construct our robot
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
collision_filter_manager = scene_graph.collision_filter_manager()
inspector = scene_graph.model_inspector()

parser = Parser(plant)
directives_file = FindResourceOrThrow("drake/C_Iris_Examples/directives/trans2dof_directives.yaml") 
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)

# Exclude the base of the iiwa from collision with the box
iiwa_base_name = "iiwa_twoDOF_link_0"
box_base_name = "box1"
def GetCollisionGeometriesByName(name):
    body = plant.GetBodyByName(name)
    return plant.GetCollisionGeometriesForBody(body)
iiwa_base_geom_set = GeometrySet()
iiwa_base_geom_set.Add(GetCollisionGeometriesByName(iiwa_base_name))
box_base_geom_set = GeometrySet()
box_base_geom_set.Add(GetCollisionGeometriesByName(box_base_name))

collision_filter_manager.Apply(
    CollisionFilterDeclaration().ExcludeBetween(iiwa_base_geom_set, box_base_geom_set)
)
plant.Finalize()

        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(plant.num_positions())

do_viz = True

# The object we will use to perform our certification.
cspace_free_polytope = CspaceFreePolytope(plant, scene_graph, SeparatingPlaneOrder.kAffine, q_star)

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity if you want
# to see the plant with the collision geometries.
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, cspace_free_polytope, viz_role=Role.kIllustration)
visualizer.visualize_collision_constraint(factor = 1.5, num_points = 100)
visualizer.meshcat_cspace.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,0,1])))
visualizer.meshcat_task_space.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([1,0,0])))

INFO:drake:Meshcat listening for connections at http://localhost:7000
INFO:drake:Meshcat listening for connections at http://localhost:7001


## Set up the sliders so we can move the plant around manually

You can use the sliders below to move the two degrees of freedom of the plant around. A green dot will appear in the TC-space visualization describing the current TC-space configuration.

In [6]:
sliders = []
for i in range(plant.num_positions()):
    q_low = plant.GetPositionLowerLimits()[i]
    q_high = plant.GetPositionUpperLimits()[i]
    sliders.append(widgets.FloatSlider(min=q_low, max=q_high, value=0, description=f"q{i}"))

q = np.zeros(plant.num_positions())
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.show_res_q(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


FloatSlider(value=0.0, description='q0', max=0.96706, min=-0.96706)

FloatSlider(value=0.0, description='q1', max=2.0944, min=-2.0944)

# Generate and Certify Regions

Around some nominal seed postures, we will grow certified regions by seeding our alternation algorithm using a small initial polytope.

In [7]:
# Some seedpoints
seed_points_q = np.array([[-0.07, 1.81],
                          [0.03, -1.79],
#                           [0.43, -1.69]
                         ])
seed_points = np.array([Ratfk.ComputeSValue(seed_points_q[idx], np.zeros((2,)))\
                        for idx in range(seed_points_q.shape[0])])
if do_viz:
    visualizer.plot_cspace_points(seed_points, "/iris_seed_points")
    


default_scale = 1e-2
L1_ball = HPolyhedron.MakeL1Ball(2)
Linf_ball = HPolyhedron.MakeBox(-np.ones(2), np.ones(2))

template_C = np.vstack([L1_ball.A(), Linf_ball.A()])
template_d = np.hstack([default_scale*L1_ball.b(), default_scale/np.sqrt(2)*Linf_ball.b()])


def make_default_polytope_at_point(seed_point):
    return HPolyhedron(template_C, template_d + template_C @ seed_point)


# colors to plot the region. Chosen for color-blind compatibility
colors_dict = {
    0: (144,144,144),
    1:(30,136,229), # bluish
    2: (255, 193, 7), # gold
    3: (0, 140, 6), # green    
}

initial_regions = [make_default_polytope_at_point(s) for s in seed_points]
visualizer.add_group_of_regions_to_visualization(initial_regions, "initial_regions",
                            colors = list(colors_dict.values()),
                            wireframe = False,
                           opacity = 0.2)
visualizer.show_res_q(q_star)

## First we set up some options for our different certification modes

In [8]:
# set up the certifier and the options for different search techniques
solver_options = SolverOptions()
# set this to 1 if you would like to see the solver output in terminal.
solver_options.SetOption(CommonSolverOption.kPrintToConsole, 0)

# The options for when we search for new planes and positivity certificates given the polytopes
find_separation_certificate_given_polytope_options = CspaceFreePolytope.FindSeparationCertificateGivenPolytopeOptions()
find_separation_certificate_given_polytope_options.num_threads = -1
find_separation_certificate_given_polytope_options.verbose = False
find_separation_certificate_given_polytope_options.solver_options = solver_options
find_separation_certificate_given_polytope_options.ignore_redundant_C = True

# The options for when we search for a new polytope given positivity certificates.
find_polytope_given_lagrangian_option = CspaceFreePolytope.FindPolytopeGivenLagrangianOptions()
find_polytope_given_lagrangian_option.solver_options = solver_options
find_polytope_given_lagrangian_option.ellipsoid_margin_cost = CspaceFreePolytope.EllipsoidMarginCost.kGeometricMean
find_polytope_given_lagrangian_option.search_s_bounds_lagrangians = True

bilinear_alternation_options = CspaceFreePolytope.BilinearAlternationOptions()
bilinear_alternation_options.max_iter = 100
bilinear_alternation_options.convergence_tol = 1e-4
bilinear_alternation_options.find_polytope_options = find_polytope_given_lagrangian_option
bilinear_alternation_options.find_lagrangian_options = find_separation_certificate_given_polytope_options

binary_search_options = CspaceFreePolytope.BinarySearchOptions()
binary_search_options.find_lagrangian_options = find_separation_certificate_given_polytope_options
binary_search_options.scale_min = 1
binary_search_options.scale_max = 50
binary_search_options.max_iter = 50

# Growing regions with bilinear alternations search

### As the initial regions are fairly small, they won't contain collisions and so can be directly fed into the bilinear alternation algorithm.

In [9]:
# We grow certified regions around each seedpoint using bilinear alternation.
bilinear_alternation_results_by_seed_point = dict.fromkeys([tuple(s) for s in seed_points])
for i, (s, initial_region) in enumerate(zip(seed_points, initial_regions)):
    print(f"starting seedpoint {i+1}/{len(initial_regions)}")
    bilinear_alternation_options.find_polytope_options.s_inner_pts = s
    result = cspace_free_polytope.SearchWithBilinearAlternation(set(), initial_region.A(),
                                                                initial_region.b(),
                                                                bilinear_alternation_options)
    bilinear_alternation_results_by_seed_point[tuple(s)] = [(HPolyhedron(cert.C, cert.d), cert) for cert in result]
    



INFO:drake:det(Q) at the beginning is 4.900500230442336e-05
INFO:drake:Iteration 0: det(Q)=0.0003673562538292721


starting seedpoint 1/2


INFO:drake:Iteration 1: det(Q)=0.0009141133082717236
INFO:drake:Iteration 2: det(Q)=0.0016603489476269496
INFO:drake:Iteration 3: det(Q)=0.002830904988217848
INFO:drake:Iteration 4: det(Q)=0.004589376311225299
INFO:drake:Iteration 5: det(Q)=0.0067442147507104245
INFO:drake:Iteration 6: det(Q)=0.009515562768865664
INFO:drake:Iteration 7: det(Q)=0.013240939832651344
INFO:drake:Iteration 8: det(Q)=0.01797856788521609
INFO:drake:Iteration 9: det(Q)=0.025808610320481495
INFO:drake:Iteration 10: det(Q)=0.03490028643506505
INFO:drake:Iteration 11: det(Q)=0.045183211993915874
INFO:drake:Iteration 12: det(Q)=0.05735564233205553
INFO:drake:Iteration 13: det(Q)=0.06546047779565303
INFO:drake:Iteration 14: det(Q)=0.07333381743029892
INFO:drake:Iteration 15: det(Q)=0.08112096702385971
INFO:drake:Iteration 16: det(Q)=0.0890071901745668
INFO:drake:Iteration 17: det(Q)=0.0963963720734065
INFO:drake:Iteration 18: det(Q)=0.10346687727206952
INFO:drake:Iteration 19: det(Q)=0.11018122020776905
INFO:drake:

starting seedpoint 2/2


INFO:drake:Iteration 1: det(Q)=0.0006288207260410793
INFO:drake:Iteration 2: det(Q)=0.0011221874341041777
INFO:drake:Iteration 3: det(Q)=0.0017457689497692918
INFO:drake:Iteration 4: det(Q)=0.0026263322595550475
INFO:drake:Iteration 5: det(Q)=0.004162219671490634
INFO:drake:Iteration 6: det(Q)=0.006397704382179459
INFO:drake:Iteration 7: det(Q)=0.01029285983876222
INFO:drake:Iteration 8: det(Q)=0.01501463325175991
INFO:drake:Iteration 9: det(Q)=0.021961703942430317
INFO:drake:Iteration 10: det(Q)=0.03162363759915458
INFO:drake:Iteration 11: det(Q)=0.047455525896536455
INFO:drake:Iteration 12: det(Q)=0.0654294101597572
INFO:drake:Iteration 13: det(Q)=0.08137362008555266
INFO:drake:Iteration 14: det(Q)=0.09546765183637862
INFO:drake:Iteration 15: det(Q)=0.10649972327947124
INFO:drake:Iteration 16: det(Q)=0.11610694902204742
INFO:drake:Iteration 17: det(Q)=0.12470597881354659
INFO:drake:Iteration 18: det(Q)=0.13244894839606264
INFO:drake:Iteration 19: det(Q)=0.13899620451347486
INFO:drake

In [10]:
# visualize the regions and corresponding certificates
for i, result in enumerate(bilinear_alternation_results_by_seed_point.values()):
    group_name = f"/bil_alt_seed_point_{i}"
    visualizer.add_group_of_regions_and_certs_to_visualization(result, group_name, 
                                                               wireframe = False, opacity = 0.2)

# Growing regions with binary search

### While the bilinear alternation scheme has the flexibility to search for fairly flexible polytopes, it can be relatively slow. We can search for larger regions faster by uniformly growing our polytopes using binary search

In [11]:
# We grow certified regions around each seedpoint using binary search.
binary_search_results_by_seed_point = dict.fromkeys([tuple(s) for s in seed_points])
for i, (s, initial_region) in enumerate(zip(seed_points, initial_regions)):
    print(f"starting seedpoint {i+1}/{len(initial_regions)}")
    time.sleep(0.2)
    cert = cspace_free_polytope.BinarySearch(set(),initial_region.A(),
                                               initial_region.b(),s,binary_search_options)
    binary_search_results_by_seed_point[tuple(s)] = [(HPolyhedron(cert.C, cert.d), cert)]
                                                     


starting seedpoint 1/2


INFO:drake:CspaceFreePolytope::BinarySearch(): scale=25.5 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=13.25 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=19.375 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=16.3125 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=14.78125 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=15.546875 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=15.1640625 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=14.97265625 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=14.876953125 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=14.8291015625 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=14.80517578125 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=14.817138671875 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=14.8231201171875 is feasi

starting seedpoint 2/2


INFO:drake:CspaceFreePolytope::BinarySearch(): scale=25.5 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=13.25 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=7.125 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=10.1875 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=8.65625 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=9.421875 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=9.0390625 is infeasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=8.84765625 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=8.943359375 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=8.9912109375 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=9.01513671875 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=9.027099609375 is feasible
INFO:drake:CspaceFreePolytope::BinarySearch(): scale=9.0330810546875 is infeasible
INFO:drake

In [12]:
# visualize the regions and corresponding certificates
for i, result in enumerate(binary_search_results_by_seed_point.values()):
    group_name = f"/bin_search_seed_point_{i}"
    visualizer.add_group_of_regions_and_certs_to_visualization(result, group_name, 
                                                               wireframe = False, opacity = 0.2)

## Combining Binary Search and Bilinear Alternation
### Of course, we can get the best of both worlds by combining the two methods

In [13]:
# we use this to back off a bit from the binary search results for numerical reasons
def scale_polytope_about_point(hpoly, s, scale):
    if not hpoly.PointInSet(s):
        raise ValueError(f"The point s must be in the HPolyhedron")
    b_scaled = scale * hpoly.b() + (1 - scale) * hpoly.A() @ s
    return HPolyhedron(hpoly.A(), b_scaled)
    

In [14]:
# Now we grow certified regions around each seedpoint using bilinear alternation 
# starting from the binary certified regions.

# There is currently a bug in Mosek that kills the kernel if you this cell.

binary_and_bilinear_certified_regions = binary_search_results_by_seed_point.copy()
for i, (s, region_start) in enumerate(binary_and_bilinear_certified_regions.items()):
    print(f"starting seedpoint {i+1}/{len(initial_regions)}")
    time.sleep(0.2)
    cur_s = np.array(s)
    bilinear_alternation_options.find_polytope_options.s_inner_pts = cur_s
    
    # We back off a little bit from the binary search solution so that we don't encounter numerical issues.
    initial_region = scale_polytope_about_point(region_start[0][0], cur_s, 0.90)
    certificates = cspace_free_polytope.SearchWithBilinearAlternation(set(),
                                                                      initial_region.A(),
                                                                      initial_region.b(), 
                                                                      bilinear_alternation_options)
    binary_and_bilinear_certified_regions[s] += [(HPolyhedron(result.C, result.d), result) for result in certificates]

starting seedpoint 1/2


INFO:drake:det(Q) at the beginning is 0.0087235307259276
INFO:drake:Iteration 0: det(Q)=0.012316640523059696
INFO:drake:Iteration 1: det(Q)=0.018888853313718056
INFO:drake:Iteration 2: det(Q)=0.026986471565249713
INFO:drake:Iteration 3: det(Q)=0.03794103902214165
INFO:drake:Iteration 4: det(Q)=0.05094013902312175
INFO:drake:Iteration 5: det(Q)=0.06727335361609212
INFO:drake:Iteration 6: det(Q)=0.08666119192659401
INFO:drake:Iteration 7: det(Q)=0.10863018848812481
INFO:drake:Iteration 8: det(Q)=0.13141075355551834
INFO:drake:Iteration 9: det(Q)=0.15231042604061687
INFO:drake:Iteration 10: det(Q)=0.1735621133245124
INFO:drake:Iteration 11: det(Q)=0.19629739778985908
INFO:drake:Iteration 12: det(Q)=0.21826639397597322
INFO:drake:Iteration 13: det(Q)=0.23344066592297988
INFO:drake:Iteration 14: det(Q)=0.24988507076435762
INFO:drake:Iteration 15: det(Q)=0.26718675016780824
INFO:drake:Iteration 16: det(Q)=0.2832159497211415
INFO:drake:Iteration 17: det(Q)=0.2954982366901911
INFO:drake:Iterat

starting seedpoint 2/2


INFO:drake:det(Q) at the beginning is 0.0032383615488816436
INFO:drake:Iteration 0: det(Q)=0.00498943591008525
INFO:drake:Iteration 1: det(Q)=0.007745849077641783
INFO:drake:Iteration 2: det(Q)=0.011305791747938531
INFO:drake:Iteration 3: det(Q)=0.01562780424466441
INFO:drake:Iteration 4: det(Q)=0.02106072670730462
INFO:drake:Iteration 5: det(Q)=0.02763241417941027
INFO:drake:Iteration 6: det(Q)=0.036992538694202076
INFO:drake:Iteration 7: det(Q)=0.04760116078672586
INFO:drake:Iteration 8: det(Q)=0.05958593912984027
INFO:drake:Iteration 9: det(Q)=0.0738873639119212
INFO:drake:Iteration 10: det(Q)=0.08979147530369101
INFO:drake:Iteration 11: det(Q)=0.10632391286773649
INFO:drake:Iteration 12: det(Q)=0.1254426904426646
INFO:drake:Iteration 13: det(Q)=0.1425245067632466
INFO:drake:Iteration 14: det(Q)=0.15761919630482074
INFO:drake:Iteration 15: det(Q)=0.16981247896787405
INFO:drake:Iteration 16: det(Q)=0.1808264707933957
INFO:drake:Iteration 17: det(Q)=0.19102450756124795
INFO:drake:Iter

In [15]:
# visualize the regions and corresponding certificates
for i, result in enumerate(binary_and_bilinear_certified_regions.values()):
    group_name = f"/bin_then_bil_search_seed_point_{i}"
    visualizer.add_group_of_regions_and_certs_to_visualization(result, group_name, 
                                                               wireframe = False, opacity = 0.2)

## Seeding and Certifying with a Stronger Heuristic
We have also implemented another, strong heuristic for proposing good initial regions based on non-linear optimization. See Appendix TODO of our paper TODO for details

In [16]:
iris_regions = []
iris_ellipses = []

iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.configuration_space_margin = 1e-3
iris_options.relative_termination_threshold = 0.001

context_for_iris = visualizer.task_space_diagram_context
for i, s in enumerate(seed_points):
    q = Ratfk.ComputeQValue(s, np.zeros((2,)))
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context_for_iris), q)
    r = IrisInRationalConfigurationSpace(plant, 
                                         plant.GetMyContextFromRoot(context_for_iris),
                                         q_star, iris_options)
    iris_regions.append(r)
    iris_ellipses.append(r.MaximumVolumeInscribedEllipsoid())
    



In [17]:
visualizer.add_group_of_regions_to_visualization(iris_regions, "/uncertified-iris", 
                                                            wireframe = False, opacity = 0.2)

### These regions tend to be very large, but typically are not completely collision free. We can use the binary search method to find a uniform shrinking of these regions to prove their safety and then again improve them with bilinear alternations.

In [18]:
binary_search_options_for_iris = CspaceFreePolytope.BinarySearchOptions()
binary_search_options_for_iris.scale_max = 1 # it is highly unlikely that the regions can grow, but we can try
binary_search_options_for_iris.find_lagrangian_options = find_separation_certificate_given_polytope_options
binary_search_options_for_iris.max_iter = 50

In [19]:
binary_search_region_certificates_for_iris = dict.fromkeys([tuple(s) for s in seed_points])
for i, (s, initial_region) in enumerate(zip(seed_points, iris_regions)):
    print(f"starting seedpoint {i+1}/{len(iris_regions)}")
    time.sleep(0.2)    
    cert = cspace_free_polytope.BinarySearch(set(),
                                                    initial_region.A(),
                                                    initial_region.b(), 
                                                    initial_region.MaximumVolumeInscribedEllipsoid().center(), 
                                                    binary_search_options_for_iris)
    binary_search_region_certificates_for_iris[tuple(s)] = [(HPolyhedron(cert.C, cert.d), cert)]

starting seedpoint 1/2


INFO:drake:CspaceFreePolytope::BinarySearch(): scale_max=1.0 is feasible.


starting seedpoint 2/2


INFO:drake:CspaceFreePolytope::BinarySearch(): scale_max=1.0 is feasible.


In [20]:
for i, result in enumerate(binary_search_region_certificates_for_iris.values()):
    group_name = f"/certified-iris-bin_seed_point_{i}"
    visualizer.add_group_of_regions_and_certs_to_visualization(result, group_name, 
                                                            wireframe = False, opacity = 0.2)

In [21]:
# Finally we grow the regions again using bilinear alternations.

# There is a numerical bug in Mosek that kills the kernel of we try this
bilinear_certified_regions_iris = binary_search_region_certificates_for_iris.copy()

# There is a numerical bug in Mosek that kills the kernel of we try this.
for i, (s, region_start) in enumerate(bilinear_certified_regions_iris.items()):
    print(f"starting seedpoint {i+1}/{len(initial_regions)}")
    time.sleep(0.2)
    cur_s = np.array(s)
    bilinear_alternation_options.find_polytope_options.s_inner_pts = cur_s
    
    # We back off a little bit from the binary search solution so that we don't encounter numerical issues.
    initial_region = scale_polytope_about_point(region_start[0][0], cur_s, 0.90)
    certificates = cspace_free_polytope.SearchWithBilinearAlternation(set(),
                                                                      initial_region.A(),
                                                                      initial_region.b(), 
                                                                      bilinear_alternation_options)
    bilinear_certified_regions_iris[s] += [(HPolyhedron(result.C, result.d), result) for result in certificates]

starting seedpoint 1/2


INFO:drake:det(Q) at the beginning is 0.30316051215037887
INFO:drake:Iteration 0: det(Q)=0.3199784045116769
INFO:drake:Iteration 1: det(Q)=0.3332768874616944
INFO:drake:Iteration 2: det(Q)=0.341609466817073
INFO:drake:Iteration 3: det(Q)=0.34990160311242585
INFO:drake:Iteration 4: det(Q)=0.35711480055004935
INFO:drake:Iteration 5: det(Q)=0.3634654439449514
INFO:drake:Iteration 6: det(Q)=0.3689944886960938
INFO:drake:Iteration 7: det(Q)=0.373802080324532
INFO:drake:Iteration 8: det(Q)=0.37787656929261615
INFO:drake:Iteration 9: det(Q)=0.38132469569517624
INFO:drake:Iteration 10: det(Q)=0.3842144544152936
INFO:drake:Iteration 11: det(Q)=0.38660563062900405
INFO:drake:Iteration 12: det(Q)=0.38857047147937857
INFO:drake:Iteration 13: det(Q)=0.3901730522555996
INFO:drake:Iteration 14: det(Q)=0.39145067403317796
INFO:drake:Iteration 15: det(Q)=0.3924571667687004
INFO:drake:Iteration 16: det(Q)=0.3932498532935734
INFO:drake:Iteration 17: det(Q)=0.3938801372446443
INFO:drake:Iteration 18: det(

starting seedpoint 2/2


INFO:drake:det(Q) at the beginning is 0.30705770639761487
INFO:drake:Iteration 0: det(Q)=0.3200283405822358
INFO:drake:Iteration 1: det(Q)=0.32842725600805156
INFO:drake:Iteration 2: det(Q)=0.33588262317586837
INFO:drake:Iteration 3: det(Q)=0.342174786707419
INFO:drake:Iteration 4: det(Q)=0.3489482207213467
INFO:drake:Iteration 5: det(Q)=0.3547242593670765
INFO:drake:Iteration 6: det(Q)=0.3596712384878305
INFO:drake:Iteration 7: det(Q)=0.3639068055858044
INFO:drake:Iteration 8: det(Q)=0.36769186175076146
INFO:drake:Iteration 9: det(Q)=0.3710578852914775
INFO:drake:Iteration 10: det(Q)=0.37391151047133014
INFO:drake:Iteration 11: det(Q)=0.37625041632352935
INFO:drake:Iteration 12: det(Q)=0.37813297511164823
INFO:drake:Iteration 13: det(Q)=0.3796309403094917
INFO:drake:Iteration 14: det(Q)=0.38084105043681904
INFO:drake:Iteration 15: det(Q)=0.38177929864855303
INFO:drake:Iteration 16: det(Q)=0.38250633565943487
INFO:drake:Iteration 17: det(Q)=0.3830723376090853
INFO:drake:Iteration 18: d

In [22]:
for i, result in enumerate(bilinear_certified_regions_iris.values()):
    visualizer.add_group_of_regions_and_certs_to_visualization(result, "/certified-iris-bin", 
                                                            wireframe = False, opacity = 0.2)

# Now it is time to play with the visualization! If you open the task space visualizer, we can highlight specific geometries and their separating planes in different regions.
## Notice that when regions overlap, we get multiple planes certifying non-collision

In [26]:
for slider in sliders:
    display(slider)

FloatSlider(value=-0.46706, description='q0', max=0.96706, min=-0.96706)

FloatSlider(value=-0.1944, description='q1', max=2.0944, min=-2.0944)

In [ ]:
visualizer.

In [24]:
# Uncomment this cell for an interface for choosing good planes of interest
# plane_indices = np.arange(0, len(cspace_free_polytope.separating_planes()))
# last_plane_index = 0
# cur_plane_index = 0
# plane_indices_to_geometries = {v: k for k,v in cspace_free_polytope.map_geometries_to_separating_planes().items()}

# while True:
#     visualizer.meshcat_task_space.Delete(f"/plane_{last_plane_index}")
#     geom1, geom2 = plane_indices_to_geometries[cur_plane_index]
#     color = Rgba(0,0,1,1)
    
#     prefix = f"/plane_{cur_plane_index}"
#     visualizer.highlight_geometry_id(geom1, color, prefix+f"/{geom1}")
#     visualizer.highlight_geometry_id(geom2, color, prefix+f"/{geom2}")
#     print(f"current plane index = {cur_plane_index}")
    
#     last_plane_index = cur_plane_index
#     valid_key_pressed = False
#     while not valid_key_pressed:
#         key_press = input("Enter d to increase plane_count, a to decrease, and e to exit...")
#         if key_press in ["a", "d", "e"]:
#             valid_key_pressed = True
#     if key_press == "a":
#         cur_plane_index = max(cur_plane_index - 1, min(plane_indices))
#     elif key_press == "d":
#         cur_plane_index = min(cur_plane_index + 1, max(plane_indices))
#     elif key_press == "e":
#         break
#     print()

In [25]:
# Uncomment this cell for seeing all the planes associated to a particular geometry id
# interesting_geometry = 65
# plane_indices = np.arange(0, len(cspace_free_polytope.separating_planes()))
# plane_indices_to_geometries = {v: k for k,v in cspace_free_polytope.map_geometries_to_separating_planes().items()}

# associated_plane_indices = []
# for plane_index, (geom1, geom2) in plane_indices_to_geometries.items():
#     if interesting_geometry in (geom1.get_value(), geom2. get_value()):
#         associated_plane_indices.append(plane_index)

        
# last_associated_index_index = 0
# cur_associated_index_index = 0

# visualizer.clear_plane_indices_of_interest()
# while True:
#     last_plane_index = associated_plane_indices[last_associated_index_index]
#     cur_plane_index = associated_plane_indices[cur_associated_index_index]
    
#     print(f"current plane index = {cur_plane_index}")
    
#     visualizer.remove_plane_indices_of_interest(last_plane_index)
#     visualizer.add_plane_indices_of_interest(cur_plane_index)
    
#     cur_q = visualizer.plant.GetPositions(visualizer.plant_context)
#     visualizer.show_res_q(cur_q)
    
#     last_associated_index_index = cur_associated_index_index
#     valid_key_pressed = False
#     while not valid_key_pressed:
#         key_press = input("Enter d to increase plane_count, a to decrease, and e to exit...")
#         if key_press in ["a", "d", "e"]:
#             valid_key_pressed = True
#     if key_press == "a":
#         cur_associated_index_index = max(cur_associated_index_index - 1, min(associated_plane_indices))
#     elif key_press == "d":
#         cur_associated_index_index = min(cur_associated_index_index + 1, max(associated_plane_indices))
#     elif key_press == "e":
#         break
#     print()